In [3]:
import torch
from torch import nn
from d2l import torch as d2l
import torch.nn.functional as F

### 1. Implement average pooling through a convolution.

In [3]:
def AveragePool2d(X, pool_size):
    p_h, p_w = pool_size
    K = torch.ones(pool_size) / p_h / p_w
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + p_h, j:j + p_w] * K).sum()
    
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
AveragePool2d(X, (2, 2))

tensor([[2., 3.],
        [5., 6.]])

### 2. Prove that max-pooling cannot be implemented through a convolution alone.

Max-pooling cannot be implemented through a convolution alone because max-pooling involves a non-linear operation that selects the maximum value within a pooling window, while convolution is a linear operation that computes a weighted sum of values within a kernel window. The non-linearity of max-pooling is essential for its behavior, and it cannot be achieved solely through linear convolution.

### 3. Max-pooling can be accomplished using ReLU operations, i.e., $\textrm{ReLU}(x) = \max(0, x)$.
    
1. Express $\max (a, b)$ by using only ReLU operations.
1. Use this to implement max-pooling by means of convolutions and ReLU layers. 
1. How many channels and layers do you need for a $2 \times 2$ convolution? How many for a $3 \times 3$ convolution?

(1) max(a, b) = ReLU(a - b) + b

(2)

In [11]:
def relu(x):
    return torch.max(x, torch.zeros_like(x))

def max_pooling_conv(input_tensor, pool_size, stride=(1,1)):
    batch_size, channels, height, width = input_tensor.size()
    (pool_size_x, pool_size_y) = pool_size
    output_shape = F.conv2d(input_tensor, torch.ones((channels, 1, pool_size_x, pool_size_y)), stride=stride, padding=0, groups=channels).shape
    output_tensor = torch.tensor([-torch.inf]*output_shape.numel()).reshape(output_shape)
    # Define the average pooling kernel
    for i in range(pool_size_x * pool_size_y):
        kernel = torch.zeros(pool_size_x * pool_size_y)
        kernel[i] = 1
        kernel = kernel.reshape(channels, 1, pool_size_x, pool_size_y)
        temp = F.conv2d(input_tensor, kernel, stride=stride, padding=0, groups=channels)
        output_tensor = relu(output_tensor - temp) + temp
    return output_tensor

# Example usage
input_tensor = torch.randn(1, 1, 4, 4)  # Batch size of 1, 3 channels, 6x6 input
output_tensor = max_pooling_conv(input_tensor, pool_size=(2,2))
print(input_tensor)
print(output_tensor)

tensor([[[[ 2.6286, -0.1056, -0.1891,  1.4409],
          [ 0.1364, -1.0369, -1.7734, -0.2590],
          [-0.7695,  0.2557, -2.5078, -0.9448],
          [ 1.3944,  0.6849, -1.9602,  1.2379]]]])
tensor([[[[ 2.6286, -0.1056,  1.4409],
          [ 0.2557,  0.2557, -0.2590],
          [ 1.3944,  0.6849,  1.2379]]]])


(3) ? I don't understand the question. What is 2x2 convolution?

### 4. What is the computational cost of the pooling layer? Assume that the input to the pooling layer is of size $c\times h\times w$, the pooling window has a shape of $p_\textrm{h}\times p_\textrm{w}$ with a padding of $(p_\textrm{h}, p_\textrm{w})$ and a stride of $(s_\textrm{h}, s_\textrm{w})$.

For one element in the output

- For average pooling: it requires $p_h \times p_w - 1$ times of additions and 1 division.
- For max pooling: it requires 1 max operation for $p_h \times p_w$ numbers.

There are $c \times [(h + s_h) / s_h] \times [(w + s_w) / s_w]$ elements in the output.

### 5. Why do you expect max-pooling and average pooling to work differently?

Max-pooling selects the maximum value from the pooling window and retains it as the representative value for that region. It is particularly effective at capturing the most prominent features within the window. It is robust to noise and variations in the data, as it prioritizes the most dominant information.

Average pooling calculates the average value of all the elements within the pooling window and uses this average as the representative value for that region.
Average pooling provides a more smoothed representation of the data. It can help capture a broader understanding of the data distribution within the window.

### 6. Do we need a separate minimum pooling layer? Can you replace it with another operation?

Because $min(x_1, x_2) = -max(-x_1, -x_2)$, we don't need a separate minimum pooling layer.

### 7. We could use the softmax operation for pooling. Why might it not be so popular?

- Softmax is sensitive to the scale of input values. When applied to a large set of values, it involves exponentiation, which can lead to numerical instability and overflow issues.
- Softmax is a normalization operation that converts input values into probabilities that sum to 1. This normalization means that the pooled values are context-dependent and relative to the other values in the pooling window. This might not always be suitable for pooling.
- Softmax pooling introduces additional computational complexity due to the exponentiation and normalization steps.